<a href="https://colab.research.google.com/github/foxtrotmike/CS909/blob/master/cnn_mnist_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Building a Convolutional Neural Network with PyTorch
By [Fayyaz Minhas](https://sites.google.com/view/fayyaz/home)


Welcome to this tutorial on building a Convolutional Neural Network (CNN) using PyTorch for digit recognition on the MNIST dataset. This guide is designed to walk you through the process step-by-step, from setting up the dataset to training and evaluating the model. Whether you're new to deep learning or looking to refine your skills, this tutorial offers insights into the practical application of CNNs using one of the most popular deep learning frameworks.

## Introduction to the MNIST Dataset
The MNIST dataset is a classic in the field of machine learning, consisting of 70,000 grayscale images of handwritten digits (0 through 9). Each image is 28x28 pixels, and the task is to classify these images into the correct digit. It's widely used for benchmarking classification algorithms.

## Setting Up the Environment
We begin by importing necessary libraries and configuring the device to use for training. We'll use CUDA if available, allowing us to take advantage of GPU acceleration for faster training:

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


## Hyperparameters and Data Loading
Next, we define our hyperparameters and load the MNIST dataset. PyTorch's torchvision module makes it easy to download and load the MNIST dataset with minimal effort. We also set up data loaders for batching and shuffling:

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False,
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


100%|██████████| 9912422/9912422 [00:00<00:00, 171620329.26it/s]

Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 94785362.93it/s]


Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 39256250.37it/s]


Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 17445539.16it/s]


Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw



## Building the CNN Model
Our CNN model consists of two convolutional layers, each followed by batch normalization, a ReLU activation function, and max pooling. The output is then passed through a fully connected layer to produce the class scores:

In [3]:

# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            #nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            #nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

## Training the Model
We train the model using a loop that iterates over our dataset for a given number of epochs. In each epoch, we perform a forward pass, calculate the loss, and update the model's weights with backpropagation:

In [4]:

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.3231
Epoch [1/5], Step [200/600], Loss: 0.1250
Epoch [1/5], Step [300/600], Loss: 0.2113
Epoch [1/5], Step [400/600], Loss: 0.1108
Epoch [1/5], Step [500/600], Loss: 0.1025
Epoch [1/5], Step [600/600], Loss: 0.0819
Epoch [2/5], Step [100/600], Loss: 0.0867
Epoch [2/5], Step [200/600], Loss: 0.1607
Epoch [2/5], Step [300/600], Loss: 0.0108
Epoch [2/5], Step [400/600], Loss: 0.0506
Epoch [2/5], Step [500/600], Loss: 0.0272
Epoch [2/5], Step [600/600], Loss: 0.0141
Epoch [3/5], Step [100/600], Loss: 0.0611
Epoch [3/5], Step [200/600], Loss: 0.0301
Epoch [3/5], Step [300/600], Loss: 0.1263
Epoch [3/5], Step [400/600], Loss: 0.0465
Epoch [3/5], Step [500/600], Loss: 0.0181
Epoch [3/5], Step [600/600], Loss: 0.0177
Epoch [4/5], Step [100/600], Loss: 0.0196
Epoch [4/5], Step [200/600], Loss: 0.0116
Epoch [4/5], Step [300/600], Loss: 0.0270
Epoch [4/5], Step [400/600], Loss: 0.0335
Epoch [4/5], Step [500/600], Loss: 0.0094
Epoch [4/5], Step [600/600], Loss:

## Evaluating the Model
After training, we evaluate the model's performance on the test set to calculate its accuracy:

In [5]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


Test Accuracy of the model on the 10000 test images: 98.82 %


## Saving the Model
Finally, we save the trained model for future use:

In [6]:
torch.save(model.state_dict(), 'model.ckpt')


This tutorial has guided you through the process of building, training, and evaluating a CNN with PyTorch on the MNIST dataset. With this foundation, you're well-equipped to tackle more complex image classification tasks and explore deeper CNN architectures.




